In [4]:
import genjax
import jax
import jax.numpy as jnp
from genjax.core.interpreters import context
import dataclasses
from genjax.typing import Any
from genjax import param, trace, Normal

key = jax.random.PRNGKey(314159)
console = genjax.pretty()

In [5]:
@genjax.gen
def _submodel():
    y1 = genjax.trace("y1", genjax.Normal)(0.0, 1.0)
    y2 = genjax.trace("y2", genjax.Normal)(0.0, 1.0)
    return y1, y2


@genjax.gen
def hierarchical_simple_normal_multiple_returns():
    y1, y2 = _submodel.inline()
    return y1, y2

In [6]:
fn = hierarchical_simple_normal_multiple_returns.simulate
key, tr = fn(key, ())
tr

BuiltinTrace
├── gen_fn
│   └── BuiltinGenerativeFunction
│       └── source
│           └── PytreeClosure
│               ├── callable
│               │   └── <function hierarchical_simple_normal_multiple_returns>
│               └── environment
│                   └── list
├── args
│   └── tuple
├── retval
│   └── tuple
│       ├──  f32[]
│       └──  f32[]
├── choices
│   └── Trie
│       ├── :y1
│       │   └── DistributionTrace
│       │       ├── gen_fn
│       │       │   └── _Normal
│       │       ├── args
│       │       │   └── tuple
│       │       │       ├── (const) 0.0
│       │       │       └── (const) 1.0
│       │       ├── value
│       │       │   └──  f32[]
│       │       └── score
│       │           └──  f32[]
│       └── :y2
│           └── DistributionTrace
│               ├── gen_fn
│               │   └── _Normal
│               ├── args
│               │   └── tuple
│               │       ├── (const) 0.0
│               │       └── (const) 1.0
│               ├── value
│               │   └──  f32[]
│               └── score
│                   └──  f32[]
├── cache
│   └── Trie
└── score
    └──  f32[]